# APIs and Datasets for Scholarly Publications 

Using [lens.org](https://www.lens.org/) API for scholarly data.

## Prerequisites

1. Clone the [GitHub repository](https://github.com/kaust-library/using_lens_org): https://github.com/kaust-library/using_lens_org
1. Create your virtual environment: `python -m venv venv`.
1. Activate your environment: `. .\venv\Scripts\activate`. (Windows platform).
1. Install the required packages: `pip install -r requirements.txt`.

## Loading the Packages and Env File

Load the packages

In [1]:
import dotenv as DE
import os as OS
import requests as RQ
import pprint as PP

You may need to create a `.env` file with your _token_ on _root_ directory of your project

```
(venv) PS C:\Users\garcm0b\Work\lens_org> cat .env
MY_TOKEN=(...)
(venv) PS C:\Users\garcm0b\Work\lens_org>
```

Make sure that your `.env` file is in the `.gitignore` file, so we will not upload your credential by accident.

In [2]:
DE.load_dotenv()
api_passwd = OS.environ['MY_TOKEN']

## Examples

Using the `requests` library to test the Lens.org API.

In [3]:
headers= {"Authorization": api_passwd, "Content-Type": "application/json"}

payload = '''
{
  "query": {
    "match": {
      "title": "Malaria"
    }
  },
  "size": 5,
  "from": 0,
  "include": [
    "title",
    "lens_id",
    "patent_citations_count"
  ],
  "sort": [
    {
      "created": "desc"
    },
    {
      "year_published": "asc"
    }
  ],
  "exclude": null,
  "scroll": null,
  "scroll_id": null
}
'''

rr = RQ.post('https://api.lens.org/scholarly/search', data=payload, headers=headers)

In [4]:
print(f"Status code of our request is ''{rr.status_code}")
PP.pprint(rr.text)

Status code of our request is ''200
('{"total":107394,"data":[{"lens_id":"065-560-633-197-40X","title":"A '
 'COMPARATIVE STUDY OF THE DIFFERENT DIAGNOSTICS OF DETECTING MALARIA AND '
 'TYPHOID FEVER"},{"lens_id":"191-605-211-943-569","title":"The effectiveness '
 'of the prevention and control methods applied towards the elimination of '
 'malaria in Botswana"},{"lens_id":"111-730-442-279-483","title":"Quantifying '
 'the impact of interventions against Plasmodium vivax malaria: a model for '
 'country-specific use"},{"lens_id":"006-242-871-547-58X","title":"Malaria '
 'Control by Mass Drug Administration with Artemisinin plus Piperaquine on '
 'Grande Comore Island, Union of '
 'Comoros"},{"lens_id":"040-464-556-289-935","title":"X marks the shot against '
 'malaria"}],"results":5}')
